In [8]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics
from sklearn.svm import SVC

In [9]:
df_train = pd.read_excel('Data\\Task1and2\\train.xlsx')
df_train.head()

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,target
0,-70,-61,-66,-53,-51,-63,-82,-57,-76,-78,-66,-66,-61,-59,-73,-75,-63,-77,B37
1,-77,-74,-71,-76,-65,-63,-66,-52,-55,-75,-72,-75,-74,-61,-64,-63,-53,-63,B61
2,-53,-38,-55,-66,-62,-62,-65,-70,-62,-52,-56,-53,-66,-68,-72,-60,-68,-77,A19
3,-72,-62,-59,-65,-65,-65,-78,-82,-83,-59,-84,-60,-64,-83,-69,-72,-95,-73,A22
4,-67,-69,-65,-63,-59,-53,-70,-72,-71,-60,-61,-57,-54,-76,-61,-66,-71,-80,A33


In [10]:
df_train.shape

(36752, 19)

In [11]:
x = df_train.drop('target', axis=1)
y = df_train['target']

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(x)
x = scaler.transform(x)

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [15]:
df_test = pd.read_excel('Data\\Task1and2\\test.xlsx')
x_test = scaler.transform(df_test)

In [21]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['linear', 'rbf']
}

svc = SVC()
svc_cv = RandomizedSearchCV(svc, param_dist, cv=5, n_iter=5, n_jobs=-1)
svc_cv.fit(x_train, y_train)

print(svc_cv.best_params_)
print(svc_cv.best_score_)

{'kernel': 'linear', 'gamma': 1, 'C': 1}
0.982449627359871


In [22]:
from sklearn.metrics import accuracy_score
model = svc_cv.best_estimator_
y_pred = model.predict(x_val)
score = accuracy_score(y_pred, y_val)
formatted_score = round(score * 100, 2)
print(f'Accuracy: {formatted_score}%')

Accuracy: 98.26%


In [23]:
model.predict(x_test[0:1]) # Predicting the first row of the test data

array(['B74'], dtype=object)

In [26]:
prediction = model.predict(x_test)
prediction = pd.DataFrame(prediction, columns=['target'])
prediction.to_csv('prediction_task2.csv')